# Section I: Building the Initial Configuration

The first step to simulating a lipid bilayer is generating the initial configuration. Here the mBuild package is used to generate a minimal stratum corneum system containing a mixture of ceramide N-hydroxy sphingosine, cholesterol, and free fatty acids. The mBuild software allows for the easy and reproducible construction of a preassembled bilayer with tunable parameters. This script will use the following packages:
- numpy
- mbuild
- mdtraj

These packages are installed using Anaconda in the cell below.

In [10]:
!conda install -y numpy 
!conda install -y mbuild 
!conda install -y mdtraj
!conda install -y py3dmol

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/parashara/miniconda/envs/workflow

  added / updated specs:
    - py3dmol


The following NEW packages will be INSTALLED:

  py3dmol            conda-forge/noarch::py3dmol-0.8.0-py_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [1]:
import mbuild as mb
import numpy as np
import mdtraj as md
from bilayer import Bilayer

/Users/parashara/miniconda/envs/dev/lib/python3.7/site-packages/hoomd/meta.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/Users/parashara/miniconda/envs/dev/lib/python3.7/site-packages/hoomd/context.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  CLOCK_START = time.clock()


Below are the parameters chosen for the system being simulated. The parameter space can be easily explored further by adjusting these values.

In [2]:
def load_molecule(filename):
    """Worker function to load the configuration of a single lipid."""
    lipid = mb.load(filename)
    lipid.translate_to([0, 0, 0])
    return lipid

prototypes = dict()

for molecule_name in ["cer", "chol", "ffa", "tip3p"]:
    filename = f"./molecules/{molecule_name}.mol2"
    molecule = load_molecule(filename)
    molecule.name = molecule_name
    prototypes.update({molecule_name : molecule})

In [3]:
# Equimolar ratio of CER, CHOL, and FFA
lipids = [(prototypes["cer"], 0.33),
          (prototypes["chol"], 0.33),
          (prototypes["ffa"], 0.34)]

tilt_angle = 10 * np.pi / 180.0 # radians
water_density = 1.0 # g/cm^3
water_mass = 18.01 # amu
n_lipids_per_edge = 6
area_per_lipid = .36 # nm^2
waters_per_lipid = 40

The system is set up using the mbuild package. The code for the `Bilayer` class can be found in `bilayer.py` located in this directory.

In [4]:
system = Bilayer(lipids,
                 ref_atoms=[77, 2, 26],
                 n_lipids_x=n_lipids_per_edge, 
                 n_lipids_y=n_lipids_per_edge,
                 area_per_lipid=area_per_lipid,
                 solvent=prototypes["tip3p"], 
                 solvent_per_lipid=waters_per_lipid,
                 solvent_density=water_density, 
                 solvent_mass=water_mass,
                 tilt=tilt_angle,
                 random_seed=77777,
                 mirror=False)

# Create box with 0.2 nm boundary
box = mb.Box(mins=[0, 0, 0],
             maxs=(system.solvent_components.boundingbox.lengths + np.array([0.2, 0.2, 0.2])))

# Translate to box center
system.translate_to(box.lengths * 0.5)

# Convert to mdTraj Trajectory and save to disk
configuration = system.to_trajectory(residues=["cer", "chol", "ffa", "tip3p"], box=box)
configuration.save("start.gro")

A render of the system is visually inspected by the user in order to validate the configuration.

In [5]:
# visualize
system.visualize()

/Users/parashara/Documents/devel/git/mbuild/mbuild/compound.py:2410: UserWarning: Guessing that "<NH1 pos=( 2.3769, 3.1044, 9.5133), 0 bonds, id: 4741735256>" is element: "N"
  atom, element))
/Users/parashara/Documents/devel/git/mbuild/mbuild/compound.py:2410: UserWarning: Guessing that "<HN1 pos=( 2.4400, 3.0916, 9.5893), 0 bonds, id: 4741736264>" is element: "H"
  atom, element))
/Users/parashara/Documents/devel/git/mbuild/mbuild/compound.py:2410: UserWarning: Guessing that "<CLN pos=( 2.4232, 3.1104, 9.3872), 0 bonds, id: 4741736600>" is element: "C"
  atom, element))
/Users/parashara/Documents/devel/git/mbuild/mbuild/compound.py:2410: UserWarning: Guessing that "<OBL pos=( 2.5431, 3.1001, 9.3671), 0 bonds, id: 4741735704>" is element: "O"
  atom, element))
/Users/parashara/Documents/devel/git/mbuild/mbuild/compound.py:2410: UserWarning: Guessing that "<CTL2 pos=( 2.3251, 3.1299, 9.2724), 0 bonds, id: 4741735480>" is element: "C"
  atom, element))
/Users/parashara/Documents/devel/g

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol